# Interactive Lecture: Using the FIDUCEO easyFCDR

In this exercise we will take a look at the FIDUCO easyFCDR - what they contain and how use them in climate variable retrieval propagating the uncertainties.

| It is structured as follows:                                                             |
| ---------------------------------------------------------------------------------------- |
|1. First look at the FIDUCEO easyFCDR file                                                |
|2. Use FCDR to perform a crude surface temperature retrieval                              |
|3. Propagate the FCDR uncertainty though this processing                                  |
| *Extension Exercises*                                                                    |

**``First import required python modules``**

To run a Python cell in the iPython notebook select it and press [shift] + [Enter]

In [ ]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

## 1. First look at the FIDUCEO easyFCDR file 

In this lecture, we will take a look at a sample easyFCDR file from the AVHRR (Advanced Very High Resolution Radiometer). The AVHRR is a scanning radiometer instrument observing in 5/6 spectral channels from the visible to the thermal infrared.

### ``i. Open the sample FCDR file``

All the FIDUCEO FCDRs are made up of netCDF files of the same format. The xarray module offers useful functionality for handling netCDF files. Let's use xarray to open a sample AVHRR netCDF file and look at it's contents.

In [ ]:
avhrr_fcdr_ds = xr.open_dataset('avhrr_sample_easyFCDR.nc')

### ``ii. Inspect FCDR file contents``

In [ ]:
print(avhrr_fcdr_ds)

### ``iii. Look at FCDR data``

#### a.  Data variables

Now let's take a look at the data inside the FCDR. To do this first import a plotting function.

In [ ]:
from plotting_functions import plot_lisbon_var

Try running/editing the below function to plot:

* Brightness temperature measurements from Channel 4 (10.7 $\mu$m).
* Independent uncertainty for measurements from Channel 4
* Common temperature measurements from Channel 4
* Structured temperature measurements from Channel 4

In [ ]:
variable_name = "Ch4"
plot_lisbon_var(var=avhrr_fcdr_ds[variable_name].values,
                lon=avhrr_fcdr_ds["longitude"].values,
                lat=avhrr_fcdr_ds["latitude"].values,
                label=avhrr_fcdr_ds[variable_name].long_name.title() + " (" + avhrr_fcdr_ds[variable_name].units + ")")

#### b. Correlation scales

* _Independent Uncertainty_ - The independent errors giving rise to the independent uncertainty are uncorrelated between pixels.



* _Common Uncertainty_ - The common errors giving rise to the common uncertainty are fully correlated between pixels for a mission.




* _Structured Uncertainty_ - The structured errors that give rise to the structured uncertainty are correlated within on orbit. The extent of the correlation depends on the separation between pixels both along track and across track. This is described by the variables `cross_element_correlation_coefficients` and `cross_line_correlation_coefficients` - let's plot these variables.

In [ ]:
plt.plot(avhrr_fcdr_ds.cross_element_correlation_coefficients.values[:,4])
plt.ylabel("Correlation Coefficient")
plt.xlabel("Cross Element Pixel Separation")
plt.title("Cross Element Correlation for Structured Uncertainty")
plt.show()
plt.close('all')

In [ ]:
plt.plot(avhrr_fcdr_ds.cross_line_correlation_coefficients.values[:,4])
plt.ylabel("Correlation Coefficient")
plt.xlabel("Cross Line Pixel Separation")
plt.title("Cross Line Correlation for Structured Uncertainty")
plt.show()
plt.close('all')

#### c. Interchannel correlation

Since retrievals typically combine data from multiple channels, understanding how errors are correlated between channels is required to properly propagate uncertainties through this processing. This information is provided in the easyFCDR separately for independent, structured and common uncertainties.

In [ ]:
print("Channel Correlation Matrix Independent:")
print(avhrr_fcdr_ds.channel_correlation_matrix_independent.values)

print("\nChannel Correlation Matrix Structured:")
print(avhrr_fcdr_ds.channel_correlation_matrix_structured.values)

print("\nChannel Correlation Matrix Common:")
print(avhrr_fcdr_ds.channel_correlation_matrix_common.values)

## 2. Use FCDR to perform a crude surface temperature retrieval

A simple approximation of the retrieval equation for surface temperature is:

$T = 2 BT_4 - BT_5$

Define a function to calculate this and plot the results.

In [ ]:
def surf_temp_calc(BT4, BT5):
    return 2* BT4 - BT5

In [ ]:
surf_temp = surf_temp_calc(avhrr_fcdr_ds.Ch4.values, avhrr_fcdr_ds.Ch5.values)

In [ ]:
plot_lisbon_var(var=surf_temp,
                lon=avhrr_fcdr_ds["longitude"].values,
                lat=avhrr_fcdr_ds["latitude"].values,
                label="Crude Surface Temperature (K)")

## 3. Propagate the FCDR uncertainty though this processing

Let's now propagate the L1 per pixel uncertainties through our retrieval algorithm. We'll do this separately for the independent, structured and common uncertainties. In each case the propagation per pixel is as follows:

$u_\mathrm{x}^2(T) = c_1^2 u^2_\mathrm{x}(BT_4) + c_2^2 u^2_\mathrm{x}(BT_5) + 2 c_1 c_2 r_{BT_4,BT_5} u_\mathrm{x}(BT_4) u_\mathrm{x}(BT_4)$

where:

* $u_x(BT_4)$ - Uncertainty for given Ch4 pixel - where $\mathrm{x}$ may be random, structured or common
* $u_x(BT_5)$ - Uncertainty for given Ch5 pixel - where $\mathrm{x}$ may be random, structured or common
* $c_1$ - sensitivity coefficient of $T$ to $BT_4$
* $c_2$ - sensitivity coefficient of $T$ to $BT_4$
* $r_{BT_4,BT_5}$ - interchannel correlation between Ch4 and Ch5


Define a function for this.

NB: This is only propagating the radiometric L1 uncertinaties to L2, many more sources of uncertainty are added due to uncertainty in the retrieval model itself.

In [ ]:
def surf_temp_unc_prop(u_BT4, u_BT5, c_1, c_2, r_BT4_BT5):
    return (c_1**2 * u_BT4**2 + c_2**2 * u_BT5**2 + 2 * c_1 * c_2 * r_BT4_BT5 * u_BT4 * u_BT5)**0.5

Evaluate sensitivity coefficients:

$c_1 = \frac{\partial T}{\partial BT_4} = 2$  
$c_2 = \frac{\partial T}{\partial BT_4} = -1$

Get interchannel correlation coefficient

$r_{BT_4,BT_5} = 0.92579997$ 

In [ ]:
surf_temp_sensitivity_BT4 = 2
surf_temp_sensitivity_BT5 = -1
channel_corr_coeff_independent = 0.0
channel_corr_coeff_structured = 0.92579997
channel_corr_coeff_common = 0.9999

### i. Evaluate the independent uncertainty for the retrieved surface temperature.

In [ ]:
u_independent_surf_temp = surf_temp_unc_prop(avhrr_fcdr_ds.u_independent_Ch4,
                                             avhrr_fcdr_ds.u_independent_Ch5,
                                             surf_temp_sensitivity_BT4,
                                             surf_temp_sensitivity_BT5,
                                             channel_corr_coeff_independent)

In [ ]:
plot_lisbon_var(var=u_independent_surf_temp,
                lon=avhrr_fcdr_ds["longitude"].values,
                lat=avhrr_fcdr_ds["latitude"].values,
                label="Independent Uncertainty of Crude Surface Temperature(K)")

### ii. Evaluate the structured uncertainty for the retrieved surface temperature.

In [ ]:
u_structured_surf_temp = surf_temp_unc_prop(avhrr_fcdr_ds.u_structured_Ch4,
                                             avhrr_fcdr_ds.u_structured_Ch5,
                                             surf_temp_sensitivity_BT4,
                                             surf_temp_sensitivity_BT5,
                                             channel_corr_coeff_structured)

In [ ]:
plot_lisbon_var(var=u_structured_surf_temp,
                lon=avhrr_fcdr_ds["longitude"].values,
                lat=avhrr_fcdr_ds["latitude"].values,
                label="Structured Uncertainty of Crude Surface Temperature (K)")

### iii. Evaluate the common uncertainty for the retrieved surface temperature.

In [ ]:
u_common_surf_temp = surf_temp_unc_prop(avhrr_fcdr_ds.u_common_Ch4,
                                        avhrr_fcdr_ds.u_common_Ch5,
                                        surf_temp_sensitivity_BT4,
                                        surf_temp_sensitivity_BT5,
                                        channel_corr_coeff_common)

In [ ]:
plot_lisbon_var(var=u_common_surf_temp,
                lon=avhrr_fcdr_ds["longitude"].values,
                lat=avhrr_fcdr_ds["latitude"].values,
                label="Common Uncertainty of Crude Surface Temperature (K)")

### iv. Evaluate the total uncertainty for the retrieved surface temperature.

In [ ]:
u_total_surf_temp = (u_independent_surf_temp**2 + u_structured_surf_temp**2 + u_common_surf_temp)**0.5

In [ ]:
plot_lisbon_var(var=u_total_surf_temp,
                lon=avhrr_fcdr_ds["longitude"].values,
                lat=avhrr_fcdr_ds["latitude"].values,
                label="Total Uncertainty of Crude Surface Temperature (K)")

NB: This is only propagating the radiometric L1 uncertinaties to L2, many more sources of uncertainty are added due to uncertainty in the retrieval model itself.

## Extension Exercises

The cell below contains the code we used in interactive lecture above. Try editing it to try the following:

1. A better function for surface temperature is the following:  
$T =-1.9 + 3.6 BT_4 - 2.6 BT_5$  
Edit the code to use this retrieval instead, propagating the uncertainties.

2. Screen for cloud using some thresholds on Ch1 and Ch2, a scene specific threshold can be judged by eye.

3. Investigate the effect on the total uncertainty of changing the interchannel error correlation for the different components of uncertainty.

In [ ]:
def surf_temp_calc(BT4, BT5):
    return -1.9 + 3.6 * BT4 - 2.6 * BT5

def surf_temp_unc_prop(u_BT4, u_BT5, c_1, c_2, r_BT4_BT5):
    return (c_1**2 * u_BT4**2 + c_2**2 * u_BT5**2 + 2 * c_1 * c_2 * r_BT4_BT5 * u_BT4 * u_BT5)**0.5

surf_temp_sensitivity_BT4 = 2
surf_temp_sensitivity_BT5 = -1
channel_corr_coeff_independent = 0.0
channel_corr_coeff_structured = 0.92579997
channel_corr_coeff_common = 0.9999

surf_temp = surf_temp
u_independent_surf_temp = surf_temp_unc_prop(avhrr_fcdr_ds.u_independent_Ch4,
                                             avhrr_fcdr_ds.u_independent_Ch5,
                                             surf_temp_sensitivity_BT4,
                                             surf_temp_sensitivity_BT5,
                                             channel_corr_coeff_independent)
u_structured_surf_temp = surf_temp_unc_prop(avhrr_fcdr_ds.u_structured_Ch4,
                                             avhrr_fcdr_ds.u_structured_Ch5,
                                             surf_temp_sensitivity_BT4,
                                             surf_temp_sensitivity_BT5,
                                             channel_corr_coeff_structured)
u_common_surf_temp = surf_temp_unc_prop(avhrr_fcdr_ds.u_common_Ch4,
                                        avhrr_fcdr_ds.u_common_Ch5,
                                        surf_temp_sensitivity_BT4,
                                        surf_temp_sensitivity_BT5,
                                        channel_corr_coeff_common)
u_total_surf_temp = (u_independent_surf_temp**2 + u_structured_surf_temp**2 + u_common_surf_temp)**0.5


plot_lisbon_var(var=u_total_surf_temp,
                lon=avhrr_fcdr_ds["longitude"].values,
                lat=avhrr_fcdr_ds["latitude"].values,
                label="Total Uncertainty Crude Surface Temperature (K)")